This notebook contains the code needed to process the data which tracks the POIS and diversity, as generated by 'generate_POIS.ipynb'
This code is part of the paper "The Importance of Being Restrained" DETAILS + CITATION INFO TO ADD HERE

In [9]:
#Basic imports and setup
import numpy as np
import pickle as pkl

sdiss = ['saturate', 'mirror', 'COTN', 'unif_resample', 'toroidal']

In [5]:
#Function for processing
def process_run(mylist, filename):
    idxs_div = [x.startswith('|') for x in mylist]
    div = [float(x[1:-1]) for x in np.array(mylist)[idxs_div]]
    idxs_fit = [x.startswith('!') for x in mylist]
    fit = [float(x[1:-1]) for x in np.array(mylist)[idxs_fit]] 
    idxs_popsize = [x.startswith('@') for x in mylist]
    popsize = [int(x[1:-1]) for x in np.array(mylist)[idxs_popsize]] 
    corr = [int(x) for x in np.array(mylist)[[(not x) and (not y) and (not z) for x,y,z in zip(idxs_div, idxs_fit, idxs_popsize)]][:-1]]
    res = {'diversity' : div, 'fitness' : fit, 'corrections' : corr, 'popsize' : popsize}
    with open(filename, 'wb') as f:
        pkl.dump(res, f)

In [15]:
#Set parameters
de_version = 'shade' # We use both shade and lshade for the paper
dim = 5 # We use dims 5 and 30 for the paper
iid = 1 # we use only iid 1 in the paper to save some computation time
log_folder_location = "logs_dict" #where to put the resulting files

In [16]:
#Read the text file and split it into an array
with open(f'logs_full/log_all_fcts_sdis_5rep_iid{iid}_{de_version}_{dim}d.txt') as f:
    full_list = np.array(f.read().splitlines())
all_arrs = np.array_split(np.array(full_list), np.where(full_list == '/')[0])[1:-1]

In [17]:
idx = 0
for fid in range(1,25):
    for sdis in sdiss:
        for rep in range(5):
            temp = all_arrs[idx][1:]
            process_run(temp, f"{log_folder_location}/{de_version}_F{fid}_{dim}D_I{iid}_{sdis}_R{rep}.pkl")
            idx = idx+1